In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Fj94nKh7G58tplylPWSgP8QFEiSNMzrEeFJaT1fXpZ1b',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'webphishingdetection-donotdelete-pr-joyfwmuqht72iv'
object_key = 'dataset_website.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,index,having_IPhaving_IP_Address,URLURL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,-1,1,1,1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,2,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,3,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,4,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,5,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [3]:
x=df.iloc[:,0:31].values
y=df.iloc[:,-1].values

In [4]:
x


array([[    1,    -1,     1, ...,     1,     1,    -1],
       [    2,     1,     1, ...,     1,     1,     1],
       [    3,     1,     0, ...,     1,     0,    -1],
       ...,
       [11053,     1,    -1, ...,     1,     0,     1],
       [11054,    -1,    -1, ...,     1,     1,     1],
       [11055,    -1,    -1, ...,    -1,     1,    -1]])

In [5]:
y

array([-1, -1, -1, ..., -1, -1, -1])

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [13]:
lr.fit(x_train,y_train)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
lr_test_pred = lr.predict(x_test)

In [23]:
lr_train_pred = lr.predict(x_train)

In [24]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [25]:
print('Training accuracy: ',accuracy_score(y_train,lr_train_pred))
print('Testing accuracy: ',accuracy_score(y_test,lr_test_pred))

Training accuracy:  0.9272953414744459
Testing accuracy:  0.9176843057440073


# IBM DEPLOYMENT

In [26]:
!pip install -U ibm-watson-machine-learning

In [27]:
from ibm_watson_machine_learning import APIClient

In [28]:
import json

# Authentic and set space

In [31]:
wml_credentials = {
         "apikey":"_R4hStSER7JyHqAVGfe-BuEwyu6RCCpRkSeLE0qJ8asn",
         "url":"https://us-south.ml.cloud.ibm.com"
}

In [36]:
wml_clients = APIClient(wml_credentials)

In [37]:
wml_clients.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------------  ------------------------
ID                                    NAME                  CREATED
1c77ca82-45e0-4a6d-b9c7-dee2c9a40405  Web phishing model-1  2022-11-23T18:43:50.075Z
------------------------------------  --------------------  ------------------------


In [38]:
SPACE_ID="1c77ca82-45e0-4a6d-b9c7-dee2c9a40405"

In [41]:
wml_clients.set.default_space(SPACE_ID)

'SUCCESS'

In [42]:
wml_clients.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

# save and deploy the model

In [43]:
import sklearn

In [45]:
sklearn.__version__

'1.0.2'

In [47]:
MODEL_NAME = 'Web phishing model-1'
DEPLOYMENT_NAME = 'Web phishing model-1'
DEMO_MODEL = lr

In [49]:
# Set Python Version
software_spec_uid = wml_clients.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [50]:
# Setup model meta
model_props = {
    wml_clients.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_clients.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_clients.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [51]:
#Save model

model_details = wml_clients.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=x_train, 
    training_target=y_train
)

In [52]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'l1',
  'schemas': {'input': [{'fields': [{'name': 'f0', 'type': 'int'},
      {'name': 'f1', 'type': 'int'},
      {'name': 'f2', 'type': 'int'},
      {'name': 'f3', 'type': 'int'},
      {'name': 'f4', 'type': 'int'},
      {'name': 'f5', 'type': 'int'},
      {'name': 'f6', 'type': 'int'},
      {'name': 'f7', 'type': 'int'},
      {'name': 'f8', 'type': 'int'},
      {'name': 'f9', 'type': 'int'},
      {'name': 'f10', 'type': 'int'},
      {'name': 'f11', 'type': 'int'},
      {'name': 'f12', 'type': 'int'},
      {'name': 'f13', 'type': 'int'},
      {'name': 'f14', 'type': 'int'},
      {'name': 'f15', 'type': 'int'},
      {'name': 'f16', 'type': 'int'},
      {'name': 'f17', 'type': 'int'},
      {'name': 'f18', 'type': 'int'},
      {'name': 'f19', 'type': 'int'},
      {'name': 'f20', 'type': 'int'},
      {'name': 'f21', 'type': 'int'},
      {'name': 'f22', 'type': 'int'},
      {'name': 'f23', 'type': 'i

In [53]:
model_id = wml_clients.repository.get_model_id(model_details)
model_id

'fbef6906-0832-46cb-b332-c43c65e355c4'

In [54]:
# Set meta
deployment_props = {
    wml_clients.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_clients.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [56]:
# Deploy
deployment = wml_clients.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)



#######################################################################################

Synchronous deployment creation for uid: 'fbef6906-0832-46cb-b332-c43c65e355c4' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='38589517-7513-41da-91d4-657aff8c42dc'
------------------------------------------------------------------------------------------------


